This is the script used for solving the pick up and delivery probelm (PDP) with reinforcment learning (RL).

-- add more description here, explain the capacity constraint and the hardness of the PDP + a general overview of the RL framework. Maybe also something on RNN.



First, we import useful packages that we will use later on.

In [1]:
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd
import torch 
import random
import itertools    

%matplotlib inline

Next, we will define some useful parameters

In [2]:
random_seed = 1234 #This is just to keep having the same results, so we can confront better
random.seed(random_seed)
number_of_clients = 10 # number of clients to be visited
# demand
max_demand = 0.5  #max (normalized) demand of a client 
min_demand = 0.01 #min (normalized) demand of a client
# Time Windows

# should we add some conditions on how to create the time windows??


To start easy, I made the following assumptions.
Some assumptions will be changed later on, some will be justified

* The grid is one by one
* The depot is always in the middle [0.5,0.5]  (This can be changed by feeding the decoder the position of the depot as first input)
* also the capacity is normalized
* distances are euclidian and are directly proportional to the travel times

The assumption that we live in a 1x1 grid comes from the following idea.
We can normalize the arcs based on hteir travel times. If we consider a grid of 1x1, then, in the worst case, it takes 2\*sqrt(2) units of time to go from the depot to the pickup and back to the depot.
For example, immagine a depot in [0,0] a pickup in [1,1] and its delivery in [0,0], this extreme case would take 2\*sqrt(2) time.

Hereafter we define two functions.
* the first function, generates a new random client
* the second one returns a random instance of the PDP (an istance of PDP is completely defined by its clients given its normalization)

these functions will be useful to generate random testing instances

In [3]:
def ComputeDistance(A,B):
        
    #round to the second decimal of the second norm of [x_1,y_1]-[x_2,y_2]
    return round(np.linalg.norm(np.array(A)-np.array(B),2),2)

class ClientClass:
    
    X_p = None    # cartesian coordinate for the pick up
    Y_p = None
    X_d = None    # cartesian coordinate for the delivery
    Y_d = None
    Demand = None
    TW_p = None   # Time Windows
    TW_d = None
    ServiceTime_p = None  # service time for pickup and delivery,
    ServiceTime_d = None  # for the time being set by default to a small value.
    Depot = None
    
    def __init__(self, max_demand, min_demand, Depot):
        self.X_p = round(random.random(), 2) # the rounding is just to keep 'reasonable' numbers
        self.Y_p = round(random.random(), 2)
        
        if Depot == True:
            self.Depot = True
            self.X_d = self.X_p
            self.Y_d = self.Y_p
            self.Demand = 0
            self.TW_p = [0, 5]
            self.TW_d = [0, 5]
            self.ServiceTime_p = 0
            self.ServiceTime_d = 0
            return
        
        self.X_d = round(random.random(), 2)
        self.Y_d = round(random.random(), 2)
        # just to avoid unreasonably close values
        while ComputeDistance([self.X_p, self.Y_p],[self.X_d, self.Y_d]) < 0.05:
            self.X_p = round(random.random(), 2) 
            self.Y_p = round(random.random(), 2)
            self.X_d = round(random.random(), 2)
            self.Y_d = round(random.random(), 2)            
        self.TW_p = [0,0]
        self.TW_d = [0,0]
        self.ServiceTime_p = 0.001 # small (irrelevant) number to avoid problem when this is zero
        self.ServiceTime_d = 0.001
        while not self.FeasibleTimeWindowsConditions(Depot):
            # the condition of the previous while is just to ensure that this client is servable
            self.TW_p[0] = round(random.uniform(0, 2*np.sqrt(2)), 2)
            self.TW_p[1] = round(random.uniform(self.TW_p[0], 2*np.sqrt(2)), 2)
            self.TW_d[0] = round(random.uniform(self.TW_p[1], 2*np.sqrt(2)), 2)
            self.TW_d[1] = round(random.uniform(self.TW_d[0], 2*np.sqrt(2)), 2)
        
        if not (self.TW_p[0]<self.TW_p[1]) and (self.TW_p[1]<self.TW_d[0]) and (self.TW_d[0]<self.TW_d[1]):
            print(self.TW_p[0])
            print(self.TW_p[1])
            print(self.TW_d[0])
            print(self.TW_d[1])
            error
        
        self.Demand = round(random.uniform(min_demand, max_demand),2)
        self.Depot = False

    def FeasibleTimeWindowsConditions(self, Depot):
    
        # from depot to pickup is time-feasible
        condition1 = ComputeDistance([self.X_p,self.Y_p],[Depot.X_p,Depot.Y_p]) + self.ServiceTime_p < self.TW_p[1]
        # from pickup to delivery is time-feasible
        distance = ComputeDistance([self.X_p,self.Y_p],[self.X_d,self.Y_d])
        condition2 = self.TW_p[0] + self.ServiceTime_p + distance + self.ServiceTime_d < self.TW_d[1]
        # service times fit into the window
        condition3A = self.TW_p[0] + self.ServiceTime_p < self.TW_p[1]
        condition3B = self.TW_d[0] + self.ServiceTime_d < self.TW_d[1]
        
        return condition1 and condition2 and condition3A and condition3B
    
def NewInstancePDP(number_of_clients, max_demand, min_demand):
    
    Clients = []
    Depot = ClientClass(max_demand, min_demand, True)
    Clients.append(Depot)
    for i in range(number_of_clients):
        Clients.append(ClientClass(max_demand, min_demand, Depot))
    
    return Clients

In the next cells we will test a greedy policy which always visits the closest node among:
* the clients who were not visited yet
* the unvisited delivery locations currently in the vehicle

To do so, we create other two classes.
One class (Route) which keeps information about the (partial) route.
The other class (Solution) which collects all the routes belonging to a solution.

In [4]:
class RouteClass:
    
    ClientsVisited = None   # array of ClientClass, states the clients already picked up AND delivered
    Distance = None         # float, states the current distance travlled
    Capacity = None         # float, states the current capacity usage
    PickedUp = None         # array of ClientClass, states the clients already picked up but NOT yet delivered
    Route = None            # array of touple [x_i,y_i] stating the order of the nodes you visit
    Depot = None
    LocationsToVisit = None # array of touple [x_i,y_i, d_i, e_i, l_i] stating the nodes still to be visited 
                            # and d_i is the capacity consumed at that node; e_i and l_i determine the time window
    
    def __init__(self):
        
        self.ClientsVisited = []
        self.Distance = 0
        self.Capacity = 0
        self.Route = []             # This takes into consideration the visited locations
        self.PickedUp = []
        self.LocationsToVisit = []  # This conider the locations still to visit
        
    def FindGreedyRoute(self, Clients, depot):
        
        self.Depot = depot
        self.Route = [[depot.X_p,depot.Y_p]]
        self.LocationsToVisit = [[depot.X_p,depot.Y_p, depot.Demand, depot.TW_p[0],depot.TW_d[1]]]  
        Reachables = self.Mask(Clients)
        
        while len(Reachables)>0:
            
            Reachables = self.GreedyOrdering(Reachables)
            client = Reachables[0]
            if client not in self.PickedUp: # it means it is a pick up
                Next = [client.X_p,client.Y_p]
                self.PickedUp.append(client)
                self.Capacity+= client.Demand
                self.LocationsToVisit.append([client.X_d,client.Y_d, -client.Demand, client.TW_d[0],client.TW_d[0]])
                for i in range(len(self.LocationsToVisit)):
                    for j in range(len(self.LocationsToVisit)):
                        if i==j:
                            continue
                        if ComputeDistance(self.LocationsToVisit[i][0:2],self.LocationsToVisit[j][0:2])==0:
                            print(i,j)
                            print(self.LocationsToVisit[i])
                            print(self.LocationsToVisit[j])
                            error
            else:                                 # while this is a delivery
                Next = [client.X_d,client.Y_d]
                self.ClientsVisited.append(client)
                self.PickedUp.remove(client)
                self.LocationsToVisit.remove([client.X_d,client.Y_d, -client.Demand, client.TW_d[0],client.TW_d[0]])
                self.Capacity-= client.Demand
            distance = ComputeDistance(self.Route[-1],Next)
            self.Route.append(Next)
            self.Distance+= distance
            Reachables = self.Mask(Clients)
        self.Route.append([depot.X_d, depot.Y_d]) # final node is again the depot
            
    def GreedyOrdering(self,Reachables):
        
        Aux1 = [[r, ComputeDistance(self.Route[-1], [r.X_d,r.Y_d])] for r in Reachables if r in self.PickedUp]
        Aux2 = [[r, ComputeDistance(self.Route[-1], [r.X_p,r.Y_p])] for r in Reachables if r not in self.PickedUp]
        Aux = Aux1+Aux2
        Aux.sort(key = lambda x: x[1])
        Aux = [a[0] for a in Aux]
        
        if len(Aux)!=len(Reachables):
            print(len(Aux1))
            print(len(Aux2))
            print('\n')
            print(len(Aux))
            print(len(Reachables))
            error
        
        return Aux
            
    def Mask(self, Clients):
        
        Unvisited = [c for c in Clients if c not in self.ClientsVisited and c not in self.PickedUp]
        ToAdd = []
        for new in Unvisited:
            if self.ExistFeasibleRoute(new):
                ToAdd.append(new)
        
        return self.PickedUp + ToAdd
    
    def ExistFeasibleRoute(self, new):
        
        loc_pu = [new.X_p, new.Y_p,  new.Demand, new.TW_p[0], new.TW_p[1]]
        loc_de = [new.X_d, new.Y_d, -new.Demand, new.TW_d[0], new.TW_d[1]]
            
        # temporarly remove the depot because it is always the last node
        self.LocationsToVisit.remove([self.Depot.X_p,self.Depot.Y_p, self.Depot.Demand, self.Depot.TW_p[0],self.Depot.TW_d[1]])
        # temporarly adding the pick up and delivery of the next client (not its demand)
        self.LocationsToVisit.append(loc_pu)
        self.LocationsToVisit.append(loc_de)
        # obtaining all the possible permutations
        Permutations = [list(p) for p in list(itertools.permutations(self.LocationsToVisit))]
        
        # adding again the depot to the locations to be visited
        self.LocationsToVisit.append([self.Depot.X_p,self.Depot.Y_p, self.Depot.Demand, self.Depot.TW_p[0],self.Depot.TW_d[1]])
        # removing the new client from the locations to visit
        self.LocationsToVisit.remove(loc_pu)
        self.LocationsToVisit.remove(loc_de)
        # removing the permutations where the pick up of the new client is after its delivery
        Permutations = [p for p in Permutations if self.FindIndex(p,loc_pu)<self.FindIndex(p,loc_de)]
        # adding the depot as last node to be visited and adding the last visited node as first node
        for p in Permutations:
            p.append([self.Depot.X_p,self.Depot.Y_p, self.Depot.Demand, self.Depot.TW_p[0],self.Depot.TW_d[1]])
            p.insert(0, self.Route[-1]+[0,0,0]) # here we insert the last visited client
                                                # with a fake zero demand 
                                                # and a non restrictive time windows
        for permutation in Permutations:
            distance = self.Distance
            capacity = self.Capacity
            for i in range(len(permutation)-1):
                capacity+= permutation[i][2]
                if capacity > 1:
                    Boolean = False
                    break
                A = permutation[i][0:2]
                B = permutation[i+1][0:2]
                distance+= ComputeDistance([A[0],A[1]],[B[0],B[1]])
                distance = max(distance, permutation[i+1][3])

                if distance > permutation[i+1][4]: # this means it violates some time window
                    Boolean = False
                    break
                Boolean = True
            if Boolean:
                break
        
        return Boolean
    
    def FindIndex(self, p,loc):
        
        # stupid fucntion because you can't use .index() 
        # since it initialies different objects in itertools.permutations
        
        for index in range(len(p)):
            if ComputeDistance(p[index],loc)==0:
                return index
        print('cannot find ',loc)
        print('within: ',p)
        print('this is an error')
        stop
        
class SolutionClass:
    
    Cost = None
    Routes = None
    Clients = None
    Depot = None
    ClientsVisited = None
    mode = None
    NN = None
    
    def __init__(self, Clients, mode, NN):
        for c in Clients:
            if c.Depot:
                self.Depot = c
                break
        self.Clients = Clients
        self.Clients.remove(self.Depot)
        self.ClientsVisited = []
        self.Routes = []
        self.Cost = 0
        self.mode = mode
        self.NN = NN
        
    def solve(self):
        
        if self.mode == 'greedy':
            return self.solveGreedy()
        else:
            if self.NN==0:
                print('if you\'re not solving it greedily, then you need to assign a neural network to NN')
                print('this is an error')
                error
            yetToCome
    
    def solveGreedy(self):
        
        ClientsToVisit = [c for c in self.Clients if c not in self.ClientsVisited]
        while len(ClientsToVisit)>0:
            newRoute = RouteClass()
            self.Routes.append(newRoute)
            newRoute.FindGreedyRoute(ClientsToVisit, self.Depot)
            self.ClientsVisited = self.ClientsVisited + newRoute.ClientsVisited
            ClientsToVisit = [c for c in self.Clients if c not in self.ClientsVisited]
        self.UpdateCost()
    
    def UpdateCost(self):
        
        self.Cost = sum([r.Distance for r in self.Routes])

In [5]:
def SolvePDP(Clients, mode, NN=0):
    Solution = SolutionClass(Clients, mode, NN=0)
    Solution.solve()
    
    return Solution

In [6]:
Clients = NewInstancePDP(number_of_clients, max_demand, min_demand)
Solution = SolvePDP(Clients, 'greedy')

print('Cost', Solution.Cost)
for r in Solution.Routes:
    print('duration', r.Distance)
        
for r in Solution.Routes:
    for c in r.ClientsVisited:
        print(Clients.index(c))
    print('\n')

Cost 8.17
duration 2.66
duration 1.9100000000000001
duration 1.54
duration 2.06
8
4
3
2
1


7
9
6


5


0




# still to do

* How to generate reasonable serving times?
* all the RL

plus, there are a few things that are not clear to me with respect to the paper we build on. 
* Why don't they compare with best know solutions? 
* How to fit an instance from the literature into something we can use?